In [1]:
#-*-coding:utf-8-*-
import sys
reload(sys)
sys.setdefaultencoding('utf-8')
import codecs
import re

In [2]:
def zero_digits(s):
    """
    Replace every digit in a string by a zero.
    """
    s[0] = re.sub('\d', '0', s[0])
    return s

In [3]:
def get_tag(t):
    if '-' not in t:
        return 'O'
    else:
        return t.split('-')

In [4]:
#tag_scheme:IOBES
def get_sentence(sent, word):
    tag = get_tag(word[-1])
    ws = ' '.join(word[0]).split()
    if tag == 'O':
        for elem in ws:
            sent.append([elem, word[1], tag])
    elif tag[0] == 'S':
        if len(word[0]) == 1:
            sent.append([word[0], word[1], word[2]])
        else:
            sent.append([ws[0], word[1], 'B-' + tag[1]])
            for _w in ws[1:-1]:
                sent.append([_w, word[1], 'I-' + tag[1]])
            sent.append([ws[-1], word[1], 'E-' + tag[1]])
    elif tag[0] == 'I':
        for _w in ws:
            sent.append([_w, word[1], 'I-' + tag[1]])
    elif tag[0] == 'B':
        sent.append([ws[0], word[1], 'B-' + tag[1]])
        for _w in ws[1:]:
            sent.append([_w, word[1], 'I-' + tag[1]])
    else: 
        if len(word[0]) == 1:
            sent.append([word[0], word[1], word[2]])
        else:
            for _w in ws[:-1]:
                sent.append([_w, word[1], 'I-' + tag[1]])
            sent.append([ws[-1], word[1], 'E-' + tag[1]])
        



In [5]:
def entity2char(path):
    sentences = []
    sentence = []
    for line in codecs.open(path, 'r', 'utf8'):
        if not line.rstrip():
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            #zero_digits(line.rstrip().split()) if zeros else 
            word = line.rstrip().split()
            assert len(word) >= 2
            get_sentence(sentence, word)
    if len(sentence) > 0:
        if 'DOCSTART' not in sentence[0][0]:
            sentences.append(sentence)
    return sentences

In [6]:
def write_char_tags(path, sentences):
    fw = codecs.open(path, 'w', encoding = 'utf-8')
    for sent in sentences:
        for elems in sent:
            fw.write(' '.join(elems) + '\n')
        fw.write('\n')
    fw.close()

In [10]:
#PER:P, LOC:L, ORG: G, O
zeros = 1
#tag_scheme = ['O','S-ORG', 'B1-ORG', 'B2-ORG', 'I-ORG', 'E-ORG']
infile = './dev_after.txt'
sentences = entity2char(infile)

outfile = './dev_charset.txt'
write_char_tags(outfile, sentences)

In [11]:
radical_path = './jian_char_radicals.txt'

radicals = {}
for line in codecs.open(radical_path, 'r', encoding = 'utf-8'):
    t = line.strip().split(':')
    radicals[t[0]] = t[-1].split()

In [11]:
def create_dico(item_list):
    """
    Create a dictionary of items from a list of list of items.
    """
    assert type(item_list) is list
    dico = {}
    for items in item_list:
        for item in items:
            if item not in dico:
                dico[item] = 1
            else:
                dico[item] += 1
    return dico

def create_mapping(dico):
    """
    Create a mapping (item to ID / ID to item) from a dictionary.
    Items are ordered by decreasing frequency.
    """
    sorted_items = sorted(dico.items(), key=lambda x: (-x[1], x[0]))
    id_to_item = {i: v[0] for i, v in enumerate(sorted_items)}
    item_to_id = {v: k for k, v in id_to_item.items()}
    return item_to_id, id_to_item

In [14]:
def char_mapping(sentences):
    """
    Create a dictionary and a mapping of chars, sorted by frequency.
    """
    chars = [[x[0] for x in s] for s in sentences]
    dico = create_dico(chars)
    dico['<UNK>'] = 10000000
    char_to_id, id_to_char = create_mapping(dico)
    print "Found %i unique chars (%i in total)" % (
        len(dico), sum(len(x) for x in chars)
    )
    return dico, char_to_id, id_to_char

def radical_mapping(sentences, radicals):
    """
    Create a dictionary and a mapping of radicals, sorted by frequency.
    """

    rads = ["".join(["".join(radicals[x[0]]) for x in s if x[0] in radicals]) for s in sentences]
    dico = create_dico(rads)
    radical_to_id, id_to_radical = create_mapping(dico)
    print "Found %i unique radicals (%i in total)" % (
        len(dico), sum(len(x) for x in rads))
    return dico, radical_to_id, id_to_radical

def tag_mapping(sentences):
    """
    Create a dictionary and a mapping of tags, sorted by frequency.
    """
    tags = [[word[-1] for word in s] for s in sentences]
    dico = create_dico(tags)
    tag_to_id, id_to_tag = create_mapping(dico)
    print "Found %i unique named entity tags" % len(dico)
    return dico, tag_to_id, id_to_tag

In [15]:
dico_chars, char_to_id, id_to_char = char_mapping(sentences)
dico_chars_train = dico_chars

radical_path = '/Users/wangwenhui/Desktop/论文/NER/FOFE_charembeddings_radicalembeddings/数据/char_radicals.txt'
radicals_dict = {}
for line in codecs.open(radical_path, 'r', encoding = 'utf-8'):
    tmp = line.strip().split(":")
    radicals_dict[tmp[0]] = tmp[-1].split()
    

In [18]:
dico_radicals, radical_to_id, id_to_radical = radical_mapping(sentences, radicals_dict)


dico_tags, tag_to_id, id_to_tag = tag_mapping(sentences)

In [24]:
def prepare_dataset(sentences, char_to_id, radical_to_id, tag_to_id, radicals_dict,lower=False):
    """
    Prepare the dataset. Return a list of lists of dictionaries containing:
        - char indexes
        - radical indexes
        - tag indexes
    """

    data = []
    for s in sentences:
        str_chars = [w[0] for w in s]
        chars = [char_to_id[w if w in char_to_id else '<UNK>']
                 for w in str_chars]
        radicals = []
        for c in str_chars:
            if c in radicals_dict:
                tmp = []
                for r in radicals_dict[c]:
                    if r in radical_to_id:
                        tmp.append(radical_to_id[r]) 
                radicals.append(tmp)   


        tags = [tag_to_id[w[-1]] for w in s]
        data.append({
            'str_chars': str_chars,
            'chars': chars,
            'radicals': radicals,
            'tags': tags
        })
    return data


In [25]:
test_data = prepare_dataset(sentences, char_to_id, radical_to_id, tag_to_id, radicals_dict)

In [26]:
len(test_data)

808

In [108]:
for x in radicals_dict[u'\u535e']:
    print x